In [1]:
#setup environment
# If you're in a fresh environment, uncomment the next lines and run once:
# !pip install yfinance bs4 html5lib lxml plotly --quiet
import warnings
warnings.filterwarnings("ignore")

using yfinance to extraxt tesla stock data

In [9]:
# Install required packages into THIS kernel
%pip install --quiet yfinance bs4 html5lib lxml plotly

Note: you may need to restart the kernel to use updated packages.


In [10]:
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd

# Create ticker and download full history
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

# Reset index and show first 5 rows
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


**Webscrabing to extract Tesla revenue**

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Download the page with Tesla revenue table
url_tsla_rev = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
                "IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm")
html_data = requests.get(url_tsla_rev).text
soup = BeautifulSoup(html_data, "html.parser")

# Extract the Tesla revenue table (quarterly/annual table that contains the word 'Tesla')
tesla_revenue = pd.read_html(str(soup), match="Tesla")[0]
tesla_revenue.columns = ["Date", "Revenue"]

# Clean revenue values and drop empty rows
tesla_revenue["Revenue"] = (
    tesla_revenue["Revenue"].astype(str).str.replace(r"[\$,]", "", regex=True).str.strip()
)
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]

# Show last 5 rows
tesla_revenue.tail(5)


,Date,Revenue
8,2013,2013
9,2012,413
10,2011,204
11,2010,117
12,2009,112


In [ ]:
use yfinance

In [11]:
import yfinance as yf
import pandas as pd

gme = yf.Ticker("GME")
gme_data = gme.history(period="max")

gme_data.reset_index(inplace=True)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


Webscraping to Extract GME Revenue Data

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_gme_rev = ("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
               "IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html")
html_data_2 = requests.get(url_gme_rev).text
soup2 = BeautifulSoup(html_data_2, "html.parser")

# Extract the GameStop revenue table
gme_revenue = pd.read_html(str(soup2), match="GameStop")[0]
gme_revenue.columns = ["Date", "Revenue"]

# Clean revenue values and drop empty rows
gme_revenue["Revenue"] = (
    gme_revenue["Revenue"].astype(str).str.replace(r"[\$,]", "", regex=True).str.strip()
)
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]

# Show last 5 rows
gme_revenue.tail(5)


,Date,Revenue
11,2009,8806
12,2008,7094
13,2007,5319
14,2006,3092
15,2005,1843


In [17]:
# make Plotly display inside the notebook
import plotly.io as pio
pio.renderers.default = "notebook_connected"   # good default for Jupyter/JupyterLab
# If it still doesn’t show, try one of these:
# pio.renderers.default = "jupyterlab"
# pio.renderers.default = "iframe"


Plot Tesla Stock Graph

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def make_graph(stock_data, revenue_data, stock_name):
    # Ensure types
    sdata = stock_data.copy()
    rdata = revenue_data.copy()
    sdata["Date"] = pd.to_datetime(sdata["Date"])
    rdata["Date"] = pd.to_datetime(rdata["Date"], errors="coerce")
    rdata["Revenue"] = pd.to_numeric(rdata["Revenue"], errors="coerce")
    rdata.dropna(subset=["Date", "Revenue"], inplace=True)

    # Limit ranges as per assignment
    stock_specific   = sdata[sdata["Date"] <= "2021-06-14"]
    revenue_specific = rdata[rdata["Date"] <= "2021-04-30"]

    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        subplot_titles=("Historical Share Price", "Historical Revenue"),
        vertical_spacing=0.3
    )

    fig.add_trace(
        go.Scatter(x=stock_specific["Date"], y=stock_specific["Close"].astype(float),
                   name="Share Price"),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=revenue_specific["Date"], y=revenue_specific["Revenue"].astype(float),
                   name="Revenue"),
        row=2, col=1
    )

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    fig.update_layout(
        title=stock_name, height=900, showlegend=False, xaxis_rangeslider_visible=True
    )
    fig.show()

# Plot Tesla
make_graph(tesla_data, tesla_revenue, "Tesla")


Plot GameStop Stock Graph

In [16]:
# Reuse the same make_graph function from Q5

# Plot GameStop
make_graph(gme_data, gme_revenue, "GameStop")
